In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/capacity/chassis.csv
/kaggle/input/capacity/workshops.csv
/kaggle/input/district/District to SSO master.csv
/kaggle/input/january/January.parquet
/kaggle/input/telemetry/telemetry_event_tml_data.csv
/kaggle/input/additional/District to SSO master.xlsx
/kaggle/input/additional/WS location details.xlsx
/kaggle/input/network-expansion/February.parquet
/kaggle/input/network-expansion/all_region_workshops.csv


In [2]:
del_cha = pd.read_csv("/kaggle/input/capacity/chassis.csv")

In [3]:
workshop = pd.read_csv("/kaggle/input/capacity/workshops.csv")

In [4]:
del_cha.head(4)

,Unnamed: 0,District,Lat,Long,count,SSO,State
0,0,Kanpur Nagar,26.450001,80.239998,193,Lucknow,Uttar Pradesh
1,1,Bulandshahr,28.469999,77.669998,379,Delhi,Uttar Pradesh
2,2,Kannauj,26.889999,79.910004,12,Lucknow,Uttar Pradesh
3,3,Unnao,26.620001,80.699997,1130,Lucknow,Uttar Pradesh


In [5]:
del_cha.drop(columns='Unnamed: 0',axis=1,inplace =True)

In [6]:
workshop.drop(columns='Unnamed: 0',axis=1,inplace =True)

In [7]:
del_cha.head(3)


,District,Lat,Long,count,SSO,State
0,Kanpur Nagar,26.450001,80.239998,193,Lucknow,Uttar Pradesh
1,Bulandshahr,28.469999,77.669998,379,Delhi,Uttar Pradesh
2,Kannauj,26.889999,79.910004,12,Lucknow,Uttar Pradesh


In [8]:
workshop.head(3)

,Division,Lattitude,Longitude,SSO,State
0,2088963-Sv&Pa-Lalitpur-GvxMot,24.731576,78.431855,Lucknow,Uttar Pradesh
1,2088398-Sv&Pa-Chitrakoot-ShyCar,25.213378,80.909587,Lucknow,Uttar Pradesh
2,2088398-Sv&Pa-Chitrakoot-ShyCar,25.213378,80.909587,Lucknow,Uttar Pradesh


In [9]:
def haversine(lonlat1, lonlat2):
     """
     Calculate the great circle distance between two points
     on the earth (specified in decimal degrees)
     """
     # convert decimal degrees to radians
     lat1, lon1 = lonlat1
     lat2, lon2 = lonlat2
     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

     #haversine formula
     dlon = lon2 - lon1
     dlat = lat2 - lat1
     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
     c = 2 * asin(sqrt(a))
     r = 6371  # Radius of earth in kilometers. Use 3956 for miles
     return c * r

In [10]:
import pandas as pd
from openpyxl import load_workbook
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
from math import ceil
import numpy as np
import folium
import warnings
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.cluster import KMeans
from statistics import mode
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from haversine import haversine
from haversine import Unit

warnings.filterwarnings('ignore')
geolocator = Nominatim(user_agent="geoapiExercises")

In [11]:
for row in tqdm(range(0, del_cha.shape[0]), desc='Calculating dist. b/w chassis'):
    dis=[]
    for div in range(0, workshop.shape[0]):
        d = haversine(del_cha[['Lat', 'Long']].iloc[row],
                      workshop[['Lattitude', 'Longitude']].iloc[div],
                      unit ='km')
        dis.append(d)
    del_cha.loc[row, 'Min Distance'] = min(dis)

Calculating dist. b/w chassis: 100%|██████████| 23579/23579 [48:42<00:00,  8.07it/s]
